# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [75]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [45]:
csvfile = ('output_data/cities.csv')
cities_df = pd.read_csv(csvfile)
cities_df = cities_df.dropna(how='any')
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.12,-134.97,296.46,86.0,100.0,3.36,PF,1.594877e+09
2,Lebu,-37.62,-73.65,283.41,90.0,100.0,3.10,CL,1.594877e+09
3,Aklavik,68.22,-135.01,289.15,59.0,75.0,5.10,CA,1.594877e+09
4,Atuona,-9.80,-139.03,299.60,77.0,1.0,7.88,PF,1.594877e+09
5,Ushuaia,-54.80,-68.30,272.15,86.0,75.0,2.10,AR,1.594877e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [46]:
#configure gmaps w/key
gmaps.configure(api_key=g_key)

#storing city locations and humidity levels
locations = cities_df[['Lat', 'Lng']]
rating = cities_df['Humidity'].astype(float)

In [47]:
#plot
fig=gmaps.figure(zoom_level=1, center = (37.09, 0.71),  map_type = "SATELLITE")

#creating heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=rating, 
                                 dissipating=False, 
                                 max_intensity=100, 
                                 point_radius=4
                                )
#applying heat layer
fig.add_layer(heat_layer)

#viewing map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [83]:
weather_df = cities_df.rename(columns={'Max Temp':'Max_Temp'})
weather_df['Fahrenheit'] = weather_df.apply(lambda row: (row.Max_Temp -273.15)*9/5+32, axis=1)
weather_df.head()
True and True
temp_df = weather_df[(weather_df['Fahrenheit'] >=69)&
                     (weather_df['Fahrenheit']<=78)&
                     (weather_df['Wind Speed']<=4)&
                     (weather_df['Cloudiness']<=20)&
                     (weather_df['Humidity']<=50)
                    ]
temp_df
                        

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Fahrenheit
16,Emerald,-23.53,148.17,295.37,22.0,0.0,3.60,AU,1.594877e+09,71.996
71,Price,39.60,-110.81,298.15,22.0,1.0,2.10,US,1.594877e+09,77.000
205,Ulaangom,49.98,92.07,294.12,35.0,20.0,2.76,MN,1.594877e+09,69.746
258,Ukiah,39.15,-123.21,296.15,49.0,1.0,3.10,US,1.594877e+09,73.400
394,Cascais,38.70,-9.42,298.15,43.0,0.0,3.44,PT,1.594877e+09,77.000
417,Kuna,43.49,-116.42,298.15,33.0,0.0,1.39,US,1.594877e+09,77.000
503,Mountain Home,43.13,-115.69,294.15,19.0,1.0,3.60,US,1.594877e+09,69.800
510,Victoria Point,-27.58,153.30,295.37,38.0,0.0,1.79,AU,1.594877e+09,71.996


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [112]:
#define new df
hotel_df = temp_df.copy()
hotel_df['Hotel Name'] = ''
hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Fahrenheit,Hotel Name
16,Emerald,-23.53,148.17,295.37,22.0,0.0,3.60,AU,1.594877e+09,71.996,
71,Price,39.60,-110.81,298.15,22.0,1.0,2.10,US,1.594877e+09,77.000,
205,Ulaangom,49.98,92.07,294.12,35.0,20.0,2.76,MN,1.594877e+09,69.746,
258,Ukiah,39.15,-123.21,296.15,49.0,1.0,3.10,US,1.594877e+09,73.400,
394,Cascais,38.70,-9.42,298.15,43.0,0.0,3.44,PT,1.594877e+09,77.000,
417,Kuna,43.49,-116.42,298.15,33.0,0.0,1.39,US,1.594877e+09,77.000,
503,Mountain Home,43.13,-115.69,294.15,19.0,1.0,3.60,US,1.594877e+09,69.800,
510,Victoria Point,-27.58,153.30,295.37,38.0,0.0,1.79,AU,1.594877e+09,71.996,


In [121]:
target_radius = 5000
target_type = 'Hotel'

                                        
#set parameter
params = {
        'radius':target_radius,
        'type':target_type,
        'key':g_key
        }
for index, row in hotel_df.iterrows():
    #define url for API
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat=row['Lat']
    lng=row['Lng']
    
    params['location'] = f"{lat},{lng}"
    
    #request
    response = requests.get(base_url, params=params)

    hotel_data=response.json()            
    pprint(hotel_data)
    
    try:
        hotel_df.loc[index,'Hotel Name']=hotel_data['results'][1]['name']
    except (KeyError, IndexError):
        print("No hotel found, omitting")

['\n'
 '<dl>\n'
 '<dt>Name</dt><dd>The Irish Village</dd>\n'
 '<dt>City</dt><dd>Emerald</dd>\n'
 '<dt>Country</dt><dd>AU</dd>\n'
 '</dl>\n',
 '\n'
 '<dl>\n'
 '<dt>Name</dt><dd>National 9 - Price River Inn</dd>\n'
 '<dt>City</dt><dd>Price</dd>\n'
 '<dt>Country</dt><dd>US</dd>\n'
 '</dl>\n',
 '\n'
 '<dl>\n'
 '<dt>Name</dt><dd>Achit Nuur Hotel & Restaurant</dd>\n'
 '<dt>City</dt><dd>Ulaangom</dd>\n'
 '<dt>Country</dt><dd>MN</dd>\n'
 '</dl>\n',
 '\n'
 '<dl>\n'
 '<dt>Name</dt><dd>Hampton Inn Ukiah</dd>\n'
 '<dt>City</dt><dd>Ukiah</dd>\n'
 '<dt>Country</dt><dd>US</dd>\n'
 '</dl>\n',
 '\n'
 '<dl>\n'
 '<dt>Name</dt><dd>Hotel Vila Galé Cascais</dd>\n'
 '<dt>City</dt><dd>Cascais</dd>\n'
 '<dt>Country</dt><dd>PT</dd>\n'
 '</dl>\n',
 '\n'
 '<dl>\n'
 '<dt>Name</dt><dd>Walgreens</dd>\n'
 '<dt>City</dt><dd>Kuna</dd>\n'
 '<dt>Country</dt><dd>US</dd>\n'
 '</dl>\n',
 '\n'
 '<dl>\n'
 '<dt>Name</dt><dd>Mountain Home Inn</dd>\n'
 '<dt>City</dt><dd>Mountain Home</dd>\n'
 '<dt>Country</dt><dd>US</dd>\n'
 '</

In [122]:
hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Fahrenheit,Hotel Name
16,Emerald,-23.53,148.17,295.37,22.0,0.0,3.60,AU,1.594877e+09,71.996,The Irish Village
71,Price,39.60,-110.81,298.15,22.0,1.0,2.10,US,1.594877e+09,77.000,National 9 - Price River Inn
205,Ulaangom,49.98,92.07,294.12,35.0,20.0,2.76,MN,1.594877e+09,69.746,Achit Nuur Hotel & Restaurant
258,Ukiah,39.15,-123.21,296.15,49.0,1.0,3.10,US,1.594877e+09,73.400,Hampton Inn Ukiah
394,Cascais,38.70,-9.42,298.15,43.0,0.0,3.44,PT,1.594877e+09,77.000,Hotel Vila Galé Cascais
417,Kuna,43.49,-116.42,298.15,33.0,0.0,1.39,US,1.594877e+09,77.000,Walgreens
503,Mountain Home,43.13,-115.69,294.15,19.0,1.0,3.60,US,1.594877e+09,69.800,Mountain Home Inn
510,Victoria Point,-27.58,153.30,295.37,38.0,0.0,1.79,AU,1.594877e+09,71.996,Bay Retreat Motel


In [123]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [125]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))